In [2]:
-Xmx4g

ERROR: Error in eval(expr, envir, enclos): object 'Xmx4g' not found


In [2]:
library('devtools')

Loading required package: usethis



In [1]:
devtools::install_github("krlmlr/ulimit")


Warning message in system(cmd):
“error in running command”
Warning message in utils::untar(tarfile, ...):
“‘/bin/gtar -xf '/tmp/RtmpVn6d44/file1b911255c67b5.tar.gz' -C '/tmp/RtmpVn6d44/remotes1b91114409fba'’ returned error code 127”


ERROR: Error: Failed to install 'ulimit' from GitHub:
  error in running command


ERROR: Error in eval(expr, envir, enclos): object 'ulimit' not found


In [6]:
memory.size(max = NA)

Warning message:
“'memory.size()' is Windows-specific”


[1] Inf

In [13]:
groups = DF[,'patientID']

In [17]:
predictor_index = 1

In [18]:
groups_c[predictor_index]

ERROR: Error in eval(expr, envir, enclos): object 'groups_c' not found


In [ ]:
RLRF = function(DF, responses, predictor_index, feat_n, RF_predictor, folds, ntrees, type_model, type_input, groups_c,scaler){
    
    groups = DF[,'patientID']
    
    group_cols = groups_c[predictor_index]
    
    #Get your response dataset
    response = responses[predictor_index]
    response_df = DF[response]
    y = response_df.values
    
    if scaler == True:
        y = QuantileTransformer().fit_transform(y)
    
    y = y[:,feat_n]
    
    #Get your predictor dataset
    if type_input == 'TSNE':
        X = np.reshape(RF_predictor[predictor_index], (68,16384))
    elif type_input == 'matrix':
        X = RF_predictor[predictor_index]
                
    #############
    
    results = []
    
    y_prediction_train = []
    y_observed_train = []

    y_prediction_test = []
    y_observed_test = []
    
    group_kfold = GroupKFold(n_splits=folds)
    
    for train_index, test_index in group_kfold.split(X, y, groups):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        

        X_train_df = pd.DataFrame(X_train)
        X_trainr = pandas2ri.py2ri(X_train_df)

        X_test_df = pd.DataFrame(X_test)
        X_testr = pandas2ri.py2ri(X_test_df)

        y_trainr = robjects.FloatVector(y_train)

        blocks = np.asarray(groups_c[predictor_index])
        blocksr = robjects.FloatVector(blocks)
            
        if type_model == 'pcLasso':
            
            blocks_l = np.where(blocks[:-1] != blocks[1:])[0]
            block_list = []
            block_list.append(np.arange(blocks_l[0]+1))
            for i in range(len(blocks_l)-1):
                block_list.append(np.array(range(blocks_l[i]+1, blocks_l[i+1]+1)))
                
            blocksr = rpy2.robjects.ListVector([(str(i), x) for i, x in enumerate(block_list)])
            
            pcLasso ="""
                function(X_trainr, X_testr, y_trainr, blocksr){
                    library(pcLasso)         
                    
                    X_trainr = as.matrix(X_trainr) 
                    X_testr = as.matrix(X_testr) 
                    
                    model <- cv.pcLasso(X_trainr, y_trainr, ratio = 0.8)
                                        
                    predictions <- predict(model, X_testr)
                    predictions
                }"""            
            
            rfunc=robjects.r(pcLasso)
            
        elif type_model == 'blockForest':
            
            blockForest ="""
            function(X_trainr, X_testr, y_trainr, blocksr){
                library(blockForest)
                blocksr = list(blocksr)
                forest_obj <- blockfor(X_trainr, y_trainr, num.trees = 100, replace = TRUE, blocks=blocksr, 
                nsets = 10, num.trees.pre = 50, splitrule="extratrees",
                                block.method = "BlockForest")
                model <- forest_obj$forest
                predictions <- predict(model, X_testr)
                predictions$predictions
            }"""
            
            rfunc=robjects.r(blockForest)
            
        y_pred_train = rfunc(X_trainr, X_trainr, y_trainr, blocksr)
        y_pred_train = np.asarray(y_pred_train)

        y_pred_test = rfunc(X_trainr, X_testr, y_trainr, blocksr)
        y_pred_test = np.asarray(y_pred_test)
                        
        y_pred_train = pd.DataFrame(y_pred_train)
        y_pred_test = pd.DataFrame(y_pred_test)

        y_train = pd.DataFrame(y_train)
        y_test  = pd.DataFrame(y_test)

        y_prediction_train.append(y_pred_train)
        y_observed_train.append(y_train)
        y_prediction_test.append(y_pred_test)
        y_observed_test.append(y_test)
    
    return pd.concat(y_prediction_train), pd.concat(y_observed_train), pd.concat(y_prediction_test),pd.concat(y_observed_test)}

In [1]:
install.packages('pcLasso')
install.packages('blockForest')

also installing the dependency ‘svd’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependency ‘RcppEigen’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library('pcLasso')
library('blockForest')

In [3]:
#Import your data
load('/home/mxenoc/workspace/butterfly/data/omics.RData')

In [11]:
#Model
folds = 10 #number of folds
features = 1 #number of features to predict
epochs = 200 #number of epochs
optimiser = 'adam' #model optimiser
loss = 'mse' #model loss
ntrees = 100
kernel_size = 2

In [10]:
#Define the predictor datasets
omics = c('rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s')

In [12]:
n_samples = 40 #same as the number of responses